# Kryptologie Lab - Übung 03 - AES
- Blockchiffre mit 128 Bit Blöcken
- Schlüssel auch 128, 192, 256 Bit
- Wir betrachten nur 128 Bit Schlüssel
- Aus Schlüssel werden 11 Rundenschlüssel erzeugt

In [168]:
import numpy as np

In [169]:
encryptGaloisMatrix = np.array([[2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2]])
decryptGaloisMatrix = np.array([[0xE,0xB,0xD,9],[9,0xE,0xB,0xD],[0xD,9,0xE,0xB],[0xB,0xD,9,0xE]])

In [170]:
with open('SBox.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))
    print("SBox")
    print(sbox)

with open('SBoxInvers.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox_inverse = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))
    print("SBox Inverse")
    print(sbox_inverse)

SBox
[[ 99 124 119 123 242 107 111 197  48   1 103  43 254 215 171 118]
 [202 130 201 125 250  89  71 240 173 212 162 175 156 164 114 192]
 [183 253 147  38  54  63 247 204  52 165 229 241 113 216  49  21]
 [  4 199  35 195  24 150   5 154   7  18 128 226 235  39 178 117]
 [  9 131  44  26  27 110  90 160  82  59 214 179  41 227  47 132]
 [ 83 209   0 237  32 252 177  91 106 203 190  57  74  76  88 207]
 [208 239 170 251  67  77  51 133  69 249   2 127  80  60 159 168]
 [ 81 163  64 143 146 157  56 245 188 182 218  33  16 255 243 210]
 [205  12  19 236  95 151  68  23 196 167 126  61 100  93  25 115]
 [ 96 129  79 220  34  42 144 136  70 238 184  20 222  94  11 219]
 [224  50  58  10  73   6  36  92 194 211 172  98 145 149 228 121]
 [231 200  55 109 141 213  78 169 108  86 244 234 101 122 174   8]
 [186 120  37  46  28 166 180 198 232 221 116  31  75 189 139 138]
 [112  62 181 102  72   3 246  14  97  53  87 185 134 193  29 158]
 [225 248 152  17 105 217 142 148 155  30 135 233 206  85

## Init Galoiskörper-Operationen

In [171]:
# Addition im Galois Körper
def gfAddition(a,b):
    return np.bitwise_xor(a,b)

# Verdopplungsfunktion
def xTimes(a):
    t = a << 1
    if a & (1 << 7) != 0:
        t = t ^ 0x1b
    return t

# Addition im Galois Körper
def gfMultiply(a,b):
    index = 0
    out = 0
    while b != 0:
        if (b & 1 == 1):
            val = a
            for i in range(index):
                val = xTimes(val)
            out = gfAddition(out, val)
        index += 1
        b = b >> 1
    return out

## Init algorithm steps

In [172]:
# Initializes Block (4x4 matrix)
# 
# Input: 16 byte as text (16 chars)
# Output: 4x4 byte matrix (row first)
def initBlock(textblock):
    chars = [ord(c) for c in textblock]
    if (len(chars) < 16):
        fill = [0 for i in range(16 - len(chars))]
        chars.extend(fill)
    return np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4),order="F")

# Reverse to initBlock
# Input: 4x4 byte matrix (row first)
# Output: 16 byte as text (16 chars)
def blockToText(textblock):
    chars = [chr(c) for c in np.nditer(textblock, order="F")]
    return "".join(chars)


# Add key on 4x4 block
# Input: 4x4 block and key as text (16 chars)
# Output: 4x4 block
def addRoundKey(textblock, key):
    chars = [ord(c) for c in key]
    key = np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4),order="F")
    return np.bitwise_xor(textblock, key)

# Local substitution with sbox
# Input: 4x4 block
# Output: 4x4 block
def subBytes(textblock, reverse=False):
    if reverse:
        box = sbox_inverse
    else:
        box = sbox
    for y in range(len(textblock)):
        for x in range(len(textblock[y])):
            # first 4 bits are row
            row = (textblock[y,x] >> 4) & 0xF
            # second 4 bits are column
            col = textblock[y,x] & 0xF
            textblock[y,x] = box[row, col]
    return textblock

# Permutation
# Shifts row i, i times to the right
def shiftRows(textblock, reverse=False):
    for i in range(4):
        if reverse:
            shiftAmount = i
        else:
            shiftAmount = -i
        textblock[i] = np.roll(textblock[i], shiftAmount)
    return textblock

# column operation of a matrix multiplication in a galois field
# Input: inputColumn (4x1) and matrix (4x4)
def subColumn(inputColumn, matrix):
    outputColumn = np.copy(inputColumn)
    for row in range(4):
        val = 0
        for column in range(4):
            val = gfAddition(val, gfMultiply(matrix[row, column], inputColumn[column]))
        outputColumn[row] = val
    return outputColumn

# matrix multiplication in the galois field
def mixColumns(textblock, reverse=False):
    if reverse:
        matrix = decryptGaloisMatrix
    else:
        matrix = encryptGaloisMatrix
    for column in range(4):
        textblock[:, column] = subColumn(textblock[:, column], matrix)
    return textblock

In [173]:
text = "ABCDEFGHIJKLMNOP"

block = initBlock(text)
print(block)
new = shiftRows(block)
print(new)
new2 = shiftRows(new, reverse = True)

[[65 69 73 77]
 [66 70 74 78]
 [67 71 75 79]
 [68 72 76 80]]
[[65 69 73 77]
 [70 74 78 66]
 [75 79 67 71]
 [80 68 72 76]]


In [174]:
## some tests
text = "ABCDEFGHIJKLMNOP"

block = initBlock(text)
initialBlock = np.array(block)
textResult = blockToText(block)
assert text == textResult

block = np.array(initialBlock)
substitutedBlock = subBytes(block)
newBlock = subBytes(substitutedBlock, reverse=True)
assert np.array_equal(initialBlock, newBlock)

block = np.array(initialBlock)
roundKeyBlock = addRoundKey(block, "9876543210987654")
newBlock = addRoundKey(roundKeyBlock, "9876543210987654")
assert np.array_equal(initialBlock, newBlock)

block = np.array(initialBlock)
block = shiftRows(block)
block = shiftRows(block, reverse=True)
assert np.array_equal(initialBlock, block)

block = np.array(initialBlock)
mixedBlock = mixColumns(block)
newBlock = mixColumns(mixedBlock, reverse=True)
assert np.array_equal(initialBlock, newBlock)

## Init algorithm process

In [175]:
# Encrypts textblock (16 chars) in 11 rounds with given keys
# Output: Encrypted textblock
def encryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[0])
    for i in range(1, 10):
        textblock = subBytes(textblock)
        textblock = shiftRows(textblock)
        textblock = mixColumns(textblock)
        textblock = addRoundKey(textblock, keys[i])
    textblock = subBytes(textblock)
    textblock = shiftRows(textblock)
    textblock = addRoundKey(textblock, keys[10])
    return blockToText(textblock)

# Decrypts textblock (16 chars) in 11 rounds with given keys
# Output: Decrypted textblock
def decryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[10])
    textblock = shiftRows(textblock, reverse=True)
    textblock = subBytes(textblock, reverse=True)
    for i in range(9, 0, -1):
        textblock = addRoundKey(textblock, keys[i])
        textblock = mixColumns(textblock, reverse=True)
        textblock = shiftRows(textblock, reverse=True)
        textblock = subBytes(textblock, reverse=True)
    textblock = addRoundKey(textblock, keys[0])
    return blockToText(textblock)

## Init block splitting

In [176]:
# Encrypts any text with given keys.
# Splits text into blocks and encrypts them separately
# returns encrypted text
def encrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([encryptBlock(block, keys) for block in blocks])

# Decrypts any text with given keys.
# Splits text into blocks and decrypts them separately
# returns decrypted text
def decrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([decryptBlock(block, keys) for block in blocks])

## Testing

In [177]:
key = ["1234567890123456" for i in range(11)]

In [178]:
text = "ABCDEFGHIJKLMNOP"
print(text)
cipher = encryptBlock(text, key)
print(cipher)
decryptedText = decryptBlock(cipher, key)
print(decryptedText)
assert text == decryptedText

ABCDEFGHIJKLMNOP
?2¯â £ûÊÈm
ABCDEFGHIJKLMNOP


In [179]:
text = "Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger."
print("Input: ", text)
cipher = encrypt(text, key)
print("cypher: ", cipher)
new_text = decrypt(cipher, key)
print("text: ", new_text)

Input:  Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger.
cypher:  ÄÛüÔí>ùGÎèw§´ì¿Ù­z	 wjy/2ç<K°Ç+ó<SúÜJõ.ÓqI ö¢NôèåË²²QQM`ÝQ.ðýwÆqÜ/uÈ¯ªÅ±Z}2Y[ò
text:  Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger.       
